In [1]:
# Given the values of a single peakon on a coarse grid, this DNN predicts the value of the peakon at a particular location.
# Essentially, given the values shown by the @ and G symbols, and a location within the ... region, it will output its
# prediction for the value at that location.
# (In terms of the gen_one_data function below, then (t, x) is the location of the G symbol and (t2, x2) is the random
# location within the ... region.)
#
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @
#
#       .........
#       .       .
# @   @ . @   @ . @   @
#       .       .
#       .       .
#       .       .
# @   @ . G   @ . @   @
#       .       .
#       .........
#
# @   @   @   @   @   @
#
#
#
# @   @   @   @   @   @

In [2]:
%run project_base.ipynb

In [3]:
# Set to False to use existing weights for the neural network.
# Set to True to continue training the weights from where we left off last time.
TRAIN = True

In [4]:
def gen_one_data():
    """Generates a random feature/label combination.
    
    Here the feature is the solution on a coarse grid and a particular point
    near the middle of that grid, and the label is the value at that point."""
        
    # Random solution to the CH equation
    c = np.random.uniform(0, 10)
    peakon = Peakon(c=c)
    # Random location
    t = np.random.uniform(0, 40)
    x = c * t + np.random.uniform(-2, 2)
    
    # Random offset from the random location that we ask for predictions at. See
    # comment at the top of the notebook for why the distribution is asymmetric.
    #
    # This value is not scaled relative to the size of the grid as we expect
    # that the predictions should be scale invariant, and we do not want the
    # network to unnecessarily learn the size of coarse_grid_sep.
    #
    # This value is just the offset; as the network has no way of knowing the
    # location of the grid then adding a translation would only confuse it.
    x_offset = np.random.uniform(-0.5, 1.5)
    t_offset = np.random.uniform(-0.5, 1.5)
    
    # Coarse grid at the location
    cg = coarse_grid((t, x))
    # Features: the solution on the coarse grid and the point to interpolate at.
    X = peakon.on_grid(cg, extra=2)
    X[-2] = t_offset
    X[-1] = x_offset
    
    # Label: the solution at the interpolation point
    y = np.array([peakon((t + t_offset * coarse_grid_sep.t, 
                          x + x_offset * coarse_grid_sep.x))])
    
    return X, y

class BilinearInterp(BilinearInterpBase):
    @classmethod
    def _predict(cls, Xi):
        # Separate the location data and the grid data
        t_offset = Xi[-2]
        x_offset = Xi[-1]
        Xi = Xi[:-2]
        # Wrapped in a list for consistency: this network just happens to only
        # be trying to predict a single label.
        return [cls._interp(Xi, (t_offset * coarse_grid_sep.t, 
                                 x_offset * coarse_grid_sep.x))]

In [5]:
model_dir = './saved_models/second_reg/'

# DNN hyperparameters
hidden_units = [500] * 5 + [300] * 5
logits = 1
drop_rate = 0.4
batch_size = 500
batch_reuse = 1  # See BatchData for an explanation of batch reuse.
steps = 20000
log_steps = 1000  # How many steps to print the current loss.

for layer in hidden_units:
    model_dir += '{}_'.format(layer)
model_dir += '{}_'.format(logits)

model_dir += 'D0{}_'.format(int(drop_rate * 10))
model_dir += 'BS{}_'.format(batch_size)
model_dir += 'BR{}'.format(batch_reuse)

k_init = tfi.truncated_normal(mean=0, stddev=0.06)
model = Sequential()
for units in hidden_units:
    model.add(dense(units=units,
                    activation=tf.nn.relu,
                    kernel_initializer=k_init))
    model.add_train(dropout(rate=drop_rate))
model.add(dense(units=logits))

In [6]:
dnn = model.compile(model_dir=model_dir, 
                    config=tfe.RunConfig(log_step_count_steps=log_steps)
                   )

INFO:tensorflow:Using config: {'_model_dir': './saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1000, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65430598d0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
train_input_fn = BatchData(gen_one_data, 
                           batch_size, 
                           batch_reuse
                          )

In [8]:
if TRAIN:
    dnn.train(input_fn=train_input_fn,
              max_steps=steps
             )

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt-14619
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 14620 into ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt.
INFO:tensorflow:loss = 0.11425025, step = 14620
INFO:tensorflow:Saving checkpoints for 15395 into ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt.
INFO:tensorflow:global_step/sec: 1.28837
INFO:tensorflow:loss = 0.11744919, step = 15620 (776.179 sec)
INFO:tensorflow:Saving checkpoints for 16160 into ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt.
INFO:tensorflow:global_step/sec: 1.2754

In [12]:
testing_data = gen_test_data(gen_one_data)

In [13]:
test_regressor(testing_data, dnn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./saved_models/second_reg/500_500_500_500_500_300_300_300_300_300_1_D04_BS500_BR1/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


{'prediction': array([[3.27869577]]),
 'actual': array([[1.62915663]]),
 'average_loss': 2.7209793662556723,
 'loss': 2.7209793662556723}

In [14]:
test_regressor(testing_data, BilinearInterp)

{'prediction': array([[1.53983718]]),
 'actual': array([[1.62915663]]),
 'average_loss': 0.00797796496544269,
 'loss': 0.00797796496544269}